# Cleaning the Data

### First import the libraries and data as a dataframe

In [2]:
import pandas as pd
import numpy as np
import re # used for pattern matching and string manipulation in date column

In [3]:
um_running_df = um_running_df = pd.read_csv("/Users/idsg/Desktop/Coding/daimil10/My_midterm/TWO_CENTURIES_OF_UM_RACES.csv")

/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/1009015574.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  um_running_df = um_running_df = pd.read_csv("/Users/idsg/Desktop/Coding/daimil10/My_midterm/TWO_CENTURIES_OF_UM_RACES.csv")


### Begin cleaning the first file from Kaggle

### This code below was utilized to pull all events located within the United States


In [4]:

usa_running_df = um_running_df[um_running_df['Event name'].str.contains("(USA)", case=True, na=False)]


/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/2744969783.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  usa_running_df = um_running_df[um_running_df['Event name'].str.contains("(USA)", case=True, na=False)]


### This filter removed all Men and Women age categories into a combined age category due to there being a column associating Male and Female

In [5]:

usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('M', '')
usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('W', '')

/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/3338947911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('M', '')
/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/3338947911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('W', '')


### This gets rid of the 'h' in the athelete peformance time to format as a datetime later


In [6]:

usa_running_df['Athlete performance'] = usa_running_df['Athlete performance'].str.replace(' h', '')

/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/243772029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_running_df['Athlete performance'] = usa_running_df['Athlete performance'].str.replace(' h', '')


### Next, I cleaned all NaNs in the birth year so they were all integers, NaNs recieved a 0


In [7]:

usa_running_df['Athlete year of birth'] = usa_running_df['Athlete year of birth'].fillna(0).astype(int)


/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/1797113307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_running_df['Athlete year of birth'] = usa_running_df['Athlete year of birth'].fillna(0).astype(int)


# The next code allowed for the creation of a true age from events that had the year of and the birth year of the athlete to provide a true age along with the age category

In [8]:


usa_running_df['true age'] = usa_running_df['Year of event'] - usa_running_df['Athlete year of birth']


/var/folders/_4/_j38q0cn0kg4pnd1jf3yvymr0000gn/T/ipykernel_85590/1292364995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_running_df['true age'] = usa_running_df['Year of event'] - usa_running_df['Athlete year of birth']


### import the extra data from the csv UM_Race_Information.csv


In [9]:

event_info_df = pd.read_csv('/Users/idsg/Desktop/Coding/daimil10/My_midterm/Midterm_Running/Data/UM_Race_Information.csv')

### merge the data sets on Event name due to 1 for 1 matches


In [10]:

merged_data_df = pd.merge(usa_running_df, event_info_df, on='Event name', how='left')

### now drop merged data that is blank and only has the 100 events we selected


In [11]:

merged_data_df = merged_data_df.dropna(subset='Distance (miles)')

### This definition turn athlete performance time into total hours


In [12]:

def time_to_hours(time_str):
    if 'd' in time_str:
        d, time = time_str.split('d ')
        h, m, s = map(int, time.split(':'))
        return int(d) * 24 + h + m / 60 + s / 3600
    else:
        h, m, s = map(int, time_str.split(':'))
        return h + m / 60 + s / 3600

merged_data_df['Hours Ran'] = merged_data_df['Athlete performance'].apply(time_to_hours)

### from the distance of the race and total hours ran we get miles per hour or speed of a racer

In [13]:


merged_data_df['mph'] = merged_data_df['Distance (miles)'] / merged_data_df['Hours Ran']


### I dropped incomplete data that would skew my results


In [14]:

merged_data_df = merged_data_df.dropna(subset=['Athlete age category'])


### State abbreviations were added to utilize choropleth and map data


In [15]:

state_abbreviation = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

merged_data_df['state_code'] = merged_data_df['State'].map(state_abbreviation)


### regex was used in the Event dates column due to dates being formatted in multiple different ways
### from this definition the dates were able to format properly and a new datetime column was created with proper date time format

In [16]:


def clean_date(date):
    # Extract the pattern dd.mm.yyyy from the string
    match = re.search(r'\b\d{2}\.\d{2}\.\d{4}\b', date)
    if match:
        return match.group(0)
    return None

# Apply the function to the 'date' column
merged_data_df['cleaned_date'] = merged_data_df['Event dates'].apply(clean_date)

# Convert to datetime
merged_data_df['datetime'] = pd.to_datetime(merged_data_df['cleaned_date'], format='%d.%m.%Y', errors='coerce')

### Look at the data and remove any columns not needed.

In [17]:
um_merged_df = merged_data_df.drop(columns=["Event distance/length", 'Event dates', 'Unnamed: 0', 'Athlete average speed', 'cleaned_date'])


### Write the finished and completed data frame to a csv

In [18]:
um_merged_df.to_csv('UM_data_cleaned.csv')

In [19]:
um_merged_df.head()

,Year of event,Event name,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete ID,true age,State,Highest Elevation (feet),Elevation Gain (feet),Average state elevation (feet),Distance (miles),Hours Ran,mph,state_code,datetime
857,2018,Chuckanut 50K (USA),410,3:36:35,"*Folsom, CA",USA,1991,M,23,12058,27,Washington,1870.0,5000.0,1700.0,31.0,3.609722,8.587918,WA,2018-03-17
858,2018,Chuckanut 50K (USA),410,3:40:57,"*Santa Fe, NM",USA,1986,M,23,12059,32,Washington,1870.0,5000.0,1700.0,31.0,3.682500,8.418194,WA,2018-03-17
859,2018,Chuckanut 50K (USA),410,3:47:43,"*San Francisco, CA",IRL,1987,M,23,12060,31,Washington,1870.0,5000.0,1700.0,31.0,3.795278,8.168045,WA,2018-03-17
860,2018,Chuckanut 50K (USA),410,3:48:13,"*Ashland, OR",USA,1988,M,23,12061,30,Washington,1870.0,5000.0,1700.0,31.0,3.803611,8.150150,WA,2018-03-17
861,2018,Chuckanut 50K (USA),410,3:50:55,"*Bend, OR",USA,1981,M,35,12062,37,Washington,1870.0,5000.0,1700.0,31.0,3.848611,8.054854,WA,2018-03-17
